### Assignment 4

In [1]:
#obvious packages
import numpy as np
import pandas as pd

#text-processing packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KDTree

#sparsematrix-packages
from scipy.sparse import csc_matrix #will give sparse
from scipy.sparse.linalg import norm  #efficient norm comps, fro is standard
from scipy.linalg.interpolative import estimate_rank #estimates rank of sparse matrix
from scipy.sparse.linalg import svds #fast svd for sparse matrices.

#other
from scipy import spatial #contains a good method for tree-search KDtree
from sklearn.neighbors import KDTree

#plotting stuff
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
#load data
data = pd.read_csv('preprocess.csv',encoding ='ISO-8859-1')
data_scrub = data.dropna()
data_scrub = data.reset_index()
#print(data_scrub) #print data_scrub just to be sure

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(data_scrub["text_pp"].tolist()) #calculates matrix, this case just count, could be Tf-idf
X=X.astype("double") #sets type as double, only necessary for int vectorizers -comment out in other case.
Xt=X.transpose()


words=set() #build dictonary of included words
for line in data_scrub["text_pp"].tolist():
    words=words.union(set(line.split()))

FileNotFoundError: File b'preprocess.csv' does not exist

In [3]:

#print("Rank is estimated to be:") #estimate the rank OBS we still cannot run for larger than 3200 since svds algos require k to be smaller than the smallest dimension.
#print(estimate_rank(np.transpose(X).toarray(),eps=1e-4))


#calculates and plot the errors for a sequence of singular value amounts.
#ks=[10,100,500,1000,1500,2000]
#errors=np.zeros([len(ks),1])
#for idx,k in enumerate(ks):
 #   u, s, vt=svds(Xt,k=k)
 #   Sigma=np.diag(s)
  #  Xhat=csc_matrix(np.matmul(np.matmul(u,Sigma),vt))
   # errors[idx]=1-norm(Xt-Xhat)/norm(Xt)
#plt.plot(errors)


In [8]:
#This calculates the necessary matrix for the chosen k and coordinates for each tweet.
ktrue=400 #add your chosen ks here
u,s,vt=svds(Xt,k=ktrue)
Sigma=np.diag(s)
Sigmainv=np.diag(1/s)
#coordinate=np.squeeze(np.asarray(np.concatenate([np.matmul(np.matmul(row.todense(),u),Sigmainv) for row in X],axis=0)))
coordinate = np.array(vt).transpose()
data_scrub = data_scrub.reset_index()
for i in range(len(coordinate)):
    coordinate[i] = coordinate[i] / np.linalg.norm(coordinate[i])


In [5]:
#querier function takes a keyword and will return the coordinates
def querier(keyw,wordlist,u,Sigmainv):
    keyw=keyw.lower()
    if(type(keyw)!=str):
        print("enter string plox")
        return None
    if(set(keyw.split(" ")).intersection(wordlist)==set()):
        print("enter allowed word, not allowed!")
        return None
    T = vectorizer.transform([keyw]).astype("double")[0]
    res = np.matmul(np.matmul(T.todense(),u),Sigmainv)
    return(res/np.linalg.norm(res))

#this will takes a queryword, calcs coorindates and then gives us the closest 20 neighbours using KDtree (quick, go fast)
test=querier("General",words,u,Sigmainv)
amount_nearest_ne=20
tree = KDTree(coordinate, metric = 'euclidean')
nearest_dist, nearest_ind = tree.query(test,k=amount_nearest_ne)
print(nearest_dist)
print(nearest_ind)

for i in nearest_ind[0]:
    print(data_scrub["text"][i])


[[0.26277746 0.38240994 0.4536602  0.76158852 0.85316078 0.91558922
  0.92362268 0.92730943 0.92840541 0.93414917 0.93850757 0.9442305
  0.94633526 0.98238978 0.9960688  1.00626236 1.0075664  1.01228928
  1.01545843 1.01569732]]
[[ 180  158  190  168 2997   81 1647  233  226   77   76 2399 1573 3169
   404 2400  125 3377 2842  587]]
'Jeff Sessions, a Fitting Selection for Attorney General'
https://t.co/LjKTkAZSFy
Just met with General Petraeus--was very impressed!
General James "Mad Dog" Mattis, who is being considered for Secretary of Defense, was very impressive yesterday. A true General's General!
and fair elections. We've accepted the outcomes when we may not have liked them, and that is what must be expected of anyone standing on a -
I only wish my wonderful daughter Tiffany could have been with us at 
Mar-a-Lago for our great election victory. She is a winner!
I spent Friday campaigning with John Kennedy, of the Great State of Louisiana, for the U.S.Senate. The election is over -

In [9]:
querylist = ["God Terrorist","Heroin Epidemic","Son Daughter","Europe NATO","Russia Putin"]
reslist = []
for j in querylist:
    res = querier(j,words,u,Sigmainv)
    tree = KDTree(coordinate, metric = 'euclidean')
    nearest_dist, nearest_ind = tree.query(res,k=amount_nearest_ne)
    reslist.append(nearest_ind)


In [10]:
for queries in reslist:
    for j in queries[0]:
        print(data_scrub["text"][j])
    print("")

Yet another terrorist attack today in Israel -- a father, shot at by a Palestinian terrorist, was killed while:
https://t.co/Cv1HzKVbiT
Crooked Hillary launched her political career by letting terrorists off the hook. #DrainTheSwampÃ¢â¬Â¦ https://t.co/BLMkZQ8svK
The terrorist who killed so many people in Germany said just before crime, "by God's will we will slaughter you pigs, I swear, we will......
A message of condolences and support regarding the terrorist attacks in Tel Aviv: 
https://t.co/iulXLEANei
In Bangladesh, hostages were immediately killed by ISIS terrorists if they were unable to cite a verse from the Koran. 20 were killed!
Hillary Clinton's weakness while she was Secretary of State, has emboldened terrorists all over the world..cont: https://t.co/E5BdTiwlur
Once again someone we were told is ok turns out to be a terrorist who wants to destroy our country &amp; its people- how did he get thru system?
Boycott all Apple products  until such time as Apple gives cellphone in